In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_experiments_for_inprocessors

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_experiments_for_inprocessors

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /Users/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


## Import dependencies

In [4]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import LawSchoolDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS

from source.experiment_interface import run_exp_iter_with_inprocessor

pip install 'aif360[LawSchoolGPA]'


## Define Input Variables

In [5]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'ADB_law_school'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIRNESS_INTERVENTION_NAME = 'ADB'
FAIR_INTERVENTION_PARAMS_LST = ['debiased_classifier']
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                     FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, 'law_school_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
    # 'session_uuid': str(uuid.uuid4()),
    'session_uuid': '0c501743-e9d5-4c52-8d94-4e1e83fcd983',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  0c501743-e9d5-4c52-8d94-4e1e83fcd983


## Initialize custom objects

In [9]:
data_loader = LawSchoolDataset()
data_loader.X_data.head()

,decile1b,decile3,lsat,ugpa,zfygpa,zgpa,fulltime,fam_inc,male,tier,race
0,10.0,10.0,44.0,3.5,1.33,1.88,1.0,5.0,0.0,4.0,White
1,5.0,4.0,29.0,3.5,-0.11,-0.57,1.0,4.0,0.0,2.0,White
2,8.0,7.0,37.0,3.4,0.63,0.37,1.0,3.0,1.0,4.0,White
3,8.0,7.0,43.0,3.3,0.67,0.34,1.0,4.0,0.0,4.0,White
4,3.0,2.0,41.0,3.3,-0.67,-1.30,1.0,4.0,0.0,5.0,White


In [10]:
data_loader.X_data.shape

(20798, 11)

## Run experiment iterations

### Experiment iteration 1

In [11]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='LawSchoolDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-08 14:55:39 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': "['debiased_classifier']",
 'model_init_seed': 100,
 'session_uuid': '0c501743-e9d5-4c52-8d94-4e1e83fcd983'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-08 14:55:40 experiment_interface.py INFO    : The dataset is preprocessed


intervention_option:  debiased_classifier
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__fulltime_1.0', 'cat__fulltime_2.0', 'cat__fam_inc_1.0',
       'cat__fam_inc_2.0', 'cat__fam_inc_3.0', 'cat__fam_inc_4.0',
       'cat__fam_inc_5.0', 'cat__tier_1.0', 'cat__tier_2.0', 'cat__tier_3.0',
       'cat__tier_4.0', 'cat__tier_5.0', 'cat__tier_6.0', 'num__decile1b',
       'num__decile3', 'num__lsat', 'num__ugpa', 'num__zfygpa', 'num__zgpa',
       'male&race_binary'],
      dtype='object')
Top indexes of an X_test in the current base flow dataset:  Int64Index([ 7102,   593, 18841,  5078, 14172,  8064, 13554, 13401, 17015,
            18446,  6938,  3450,  9375, 19994, 16100,  4401,   142, 15143,
             2188,  4332],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([ 7102,   593, 18841,  5078, 14172,  8064, 13554, 13401, 17015,
            18446,  6938,  3450,  9375, 19994, 16100,  4401,   142, 15143,
             2188,  4332]

...

epoch 0; iter: 0; batch classifier loss: 0.730018; batch adversarial loss: 0.833528
epoch 1; iter: 0; batch classifier loss: 0.308060; batch adversarial loss: 0.880607
epoch 2; iter: 0; batch classifier loss: 0.237093; batch adversarial loss: 0.770558
epoch 3; iter: 0; batch classifier loss: 0.217987; batch adversarial loss: 0.690080
epoch 4; iter: 0; batch classifier loss: 0.182140; batch adversarial loss: 0.571200
epoch 5; iter: 0; batch classifier loss: 0.218811; batch adversarial loss: 0.504431
epoch 6; iter: 0; batch classifier loss: 0.198824; batch adversarial loss: 0.462173
epoch 7; iter: 0; batch classifier loss: 0.231496; batch adversarial loss: 0.418298
epoch 8; iter: 0; batch classifier loss: 0.234650; batch adversarial loss: 0.367101
epoch 9; iter: 0; batch classifier loss: 0.239843; batch adversarial loss: 0.395486
epoch 10; iter: 0; batch classifier loss: 0.129563; batch adversarial loss: 0.320923
epoch 11; iter: 0; batch classifier loss: 0.233457; batch adversarial loss:

2024-01-08 14:59:46.701737: W tensorflow/c/c_api.cc:305] Operation '{name:'39b915de-ae25-11ee-bc15-ae7d8bf09116/39b915de-ae25-11ee-bc15-ae7d8bf09116/adversary_model/b2/Adam_1/Assign' id:14197 op device:{requested: '', assigned: ''} def:{{{node 39b915de-ae25-11ee-bc15-ae7d8bf09116/39b915de-ae25-11ee-bc15-ae7d8bf09116/adversary_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](39b915de-ae25-11ee-bc15-ae7d8bf09116/39b915de-ae25-11ee-bc15-ae7d8bf09116/adversary_model/b2/Adam_1, 39b915de-ae25-11ee-bc15-ae7d8bf09116/39b915de-ae25-11ee-bc15-ae7d8bf09116/adversary_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


epoch 0; iter: 0; batch classifier loss: 0.658392; batch adversarial loss: 0.557374
epoch 1; iter: 0; batch classifier loss: 0.294416; batch adversarial loss: 0.467630
epoch 2; iter: 0; batch classifier loss: 0.276471; batch adversarial loss: 0.406063
epoch 3; iter: 0; batch classifier loss: 0.299274; batch adversarial loss: 0.390990
epoch 4; iter: 0; batch classifier loss: 0.229801; batch adversarial loss: 0.310087
epoch 5; iter: 0; batch classifier loss: 0.203792; batch adversarial loss: 0.295117
epoch 6; iter: 0; batch classifier loss: 0.277957; batch adversarial loss: 0.308744
epoch 7; iter: 0; batch classifier loss: 0.309148; batch adversarial loss: 0.345995
epoch 8; iter: 0; batch classifier loss: 0.239403; batch adversarial loss: 0.261856
epoch 9; iter: 0; batch classifier loss: 0.243664; batch adversarial loss: 0.325299
epoch 10; iter: 0; batch classifier loss: 0.217376; batch adversarial loss: 0.269087
epoch 11; iter: 0; batch classifier loss: 0.255297; batch adversarial loss:

2024-01-08 15:00:00.633140: W tensorflow/c/c_api.cc:305] Operation '{name:'39b91606-ae25-11ee-bc15-ae7d8bf09116/39b91606-ae25-11ee-bc15-ae7d8bf09116/adversary_model/b2/Adam_1/Assign' id:14987 op device:{requested: '', assigned: ''} def:{{{node 39b91606-ae25-11ee-bc15-ae7d8bf09116/39b91606-ae25-11ee-bc15-ae7d8bf09116/adversary_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](39b91606-ae25-11ee-bc15-ae7d8bf09116/39b91606-ae25-11ee-bc15-ae7d8bf09116/adversary_model/b2/Adam_1, 39b91606-ae25-11ee-bc15-ae7d8bf09116/39b91606-ae25-11ee-bc15-ae7d8bf09116/adversary_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


epoch 0; iter: 0; batch classifier loss: 0.697510; batch adversarial loss: 0.821781
epoch 1; iter: 0; batch classifier loss: 0.316984; batch adversarial loss: 0.792675
epoch 2; iter: 0; batch classifier loss: 0.210076; batch adversarial loss: 0.683848
epoch 3; iter: 0; batch classifier loss: 0.200424; batch adversarial loss: 0.582443
epoch 4; iter: 0; batch classifier loss: 0.246268; batch adversarial loss: 0.522641
epoch 5; iter: 0; batch classifier loss: 0.191786; batch adversarial loss: 0.466231
epoch 6; iter: 0; batch classifier loss: 0.275744; batch adversarial loss: 0.391277
epoch 7; iter: 0; batch classifier loss: 0.222185; batch adversarial loss: 0.373101
epoch 8; iter: 0; batch classifier loss: 0.225707; batch adversarial loss: 0.356512
epoch 9; iter: 0; batch classifier loss: 0.351194; batch adversarial loss: 0.399525
epoch 10; iter: 0; batch classifier loss: 0.271632; batch adversarial loss: 0.330535
epoch 11; iter: 0; batch classifier loss: 0.190482; batch adversarial loss:

### Experiment iteration 2

In [4]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

NameError: name 'EXPERIMENT_SEEDS' is not defined

In [21]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='LawSchoolDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-04 17:15:39 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 200,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 17:15:39 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 17:15:39 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [11]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='LawSchoolDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-04 19:29:01 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 300,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 19:29:01 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 19:29:01 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [13]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [14]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='LawSchoolDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-04 20:15:17 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 400,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 20:15:17 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 20:15:17 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 5

In [11]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='LawSchoolDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-05 03:55:49 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'experiment_iteration': 'Exp_iter_5',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 500,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-05 03:55:49 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-05 03:55:49 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 6

In [13]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [14]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='LawSchoolDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-05 04:49:43 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 600,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-05 04:49:43 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-05 04:49:43 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]